In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname(""), ".."))
import custom
import numpy as np

import json
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [2]:
# 단어 불러오기

with open("data/eng_word.json", mode = "r", encoding="UTF8") as f:
    eng_word = json.load(f)

with open("data/kor_word.json", mode = "r", encoding="UTF8") as f:
    kor_word = json.load(f)


print(len(eng_word))
print(len(kor_word))

2890
2871


In [3]:
# 데이터 불러오기

data = pd.read_csv("data/data.csv", encoding = "UTF8", sep = "\t")

display(data)

,eng,eng_len,kor,kor_len
0,go .,2,가 어 . <eos>,4
1,hi .,2,안녕 . <eos>,3
2,run !,2,뛰 어 ! <eos>,4
3,run .,2,뛰 어 . <eos>,4
4,who ?,2,누구 ? <eos>,3
...,...,...,...,...
5438,there are some important questions that scienc...,10,과학 이 답변 하 ㄹ 수 없 는 몇 가지 중요 하 ㄴ 질문 들 이 있 어 . <eos>,20
5439,company regulations mandate that workers use p...,9,회사 내규 에 의하 면 근로자 가 보호 안경 을 사용 하 도록 되 어 있 다 . <...,19
5440,company regulations mandate that workers use p...,9,회사 규칙 에 따르 면 근로자 가 보호 안경 을 쓰 도록 되 어 있 어 . <eos>,18
5441,company regulations mandate that workers use p...,9,회사 방침 에 따르 면 근로자 가 보호 안경 을 사용 하 도록 되 어 있 어 요 ....,20


In [4]:
# 데이터 x, t로 나누고, 벡터화
kor_len = data['kor_len'].max()
eng_len = data['eng_len'].max()

data_x = []
for i in range(len(data)) :
    words = data.iloc[i,2].split()
    label = custom.word_vectorize(words, kor_word, kor_len, padding_front=False)
    data_x.append(label)
data_x = np.array(data_x)
print(data_x.shape)

data_t = []
for i in range(len(data)) :
    words = data.iloc[i,0].split()
    label = custom.word_vectorize(words, eng_word, eng_len, padding_front=False)
    data_t.append(label)
data_t = np.array(data_t)
print(data_t.shape)

(5443, 20)
(5443, 10)


In [5]:
# DataLoader로 나누기
device = "cuda" if torch.cuda.is_available() else "cpu"

tensor_x = torch.tensor(data_x, dtype = torch.long, device = device)
tensor_t = torch.tensor(data_t, dtype = torch.long, device = device)

dataloader = DataLoader(list(zip(tensor_x, tensor_t)), batch_size=100, shuffle=True)
test_dataloader = DataLoader(list(zip(tensor_x, tensor_t)), batch_size=100, shuffle=False)
print(len(dataloader))
print(len(test_dataloader))

55
55


In [6]:
import torch
import torch.nn as nn

### Encoder와 Decoder 구현 하기 (함수 만들기)

class Encoder(nn.Module) :
    def __init__(self, word_len, vector_size) :
        super().__init__()
        self.embedding = nn.Embedding(word_len, vector_size, padding_idx=0) # 벡터화 함수, b값이 없는 선형함수, 차원 = (단어갯수, 벡터갯수)
        #.from_pretrained : W값을 해당 벡터로 만들겠다
        self.rnn = nn.LSTM(vector_size, vector_size, batch_first = True, bidirectional=True)# RNN 함수
    def forward(self, x) :
        x = self.embedding(x)
        y, hc = self.rnn(x) #LSTM은 hc
        return y, hc

class Attention(nn.Module) : #Value를 구하기 위한 재료,
    def __init__(self, h_size, bidirectional = False) : # h_size : 벡터 크기
        super().__init__()
        factor = 2 if bidirectional else 1
        self.U = nn.Linear(h_size * factor, h_size * factor) # query(decoder h값) 가공, query.shape = [b, 1, h_size * 2]
        self.W = nn.Linear(h_size * factor, h_size * factor) # key(encoder 출력값) 가공, key.shape = [b, f, h_size * 2]
        self.V = nn.Linear(h_size * factor, 1) # score(query & key) 계산
    def forward(self, query, key) :
        score = self.V(torch.tanh(self.U(query) + self.W(key))) # score.shape = [b, f, 1]
        score = score.permute(0,2,1) # score.shape = [b,1,f]

        weight = torch.softmax(score, dim = -1) # softmax는 함수 차원을 바꾸지 않습니다
        context = torch.bmm(weight, key) # [b,1,f] * [b,f,h_size * 2] = [b,1,h_size * 2]
        return context, weight

class Decoder(nn.Module) :
    def __init__(self, word_len, vector_size, max_len) :
        super().__init__()
        self.embedding = nn.Embedding(word_len, vector_size, padding_idx=0) #벡터화 함수
        self.attention = Attention(vector_size, bidirectional = True)
        self.rnn = nn.LSTM(vector_size * 3, vector_size, batch_first = True, bidirectional=True) # RNN 함수
        self.f = nn.Linear(vector_size * 2, word_len) #벡터 값을 다시 단어 라벨값으로 역산하는 함수
        self.max_len = max_len #문장 단어의 최대 갯수
    def forward(self, encoder_output, encoder_hc, t = None) :        
        decoder_y_list = [] #출력값
        weight_list = []

        decoder_x = torch.zeros((encoder_output.shape[0],1)).type(torch.long).to(encoder_output.device) #첫번째 입력값, 라벨값이기 때문에 차원 = (문장 갯수,1), 문장 갯수는 encoder_output에서 가져옵니다
        #.type(torch.long) 은 형변환, .to(encoder_output.device) 은 기기 배정
        decoder_hc = encoder_hc #첫번째 hc값

        for i in range(self.max_len) :
            decoder_y, decoder_hc, weight = self.forward_cal(decoder_x, decoder_hc, encoder_output) # RNN 계산 한블록 합니다.

            if t is not None : #teaching force, t가 존재할 때
                decoder_x = t[:,i:i+1]  # i번째 t의 단어 가져옴
            else : # greedy, t가 존재하지 않을 때
                decoder_x = torch.argmax(decoder_y, dim = -1).detach() #출력 값 중에 가장 큰값(라벨값) 가져오고, 해당 값은 미분 대상이 아님 (.detach())

            decoder_y_list.append(decoder_y) #출력값에 단어 하나하나 씩 저장
            weight_list.append(weight)

        decoder_y_list = torch.cat(decoder_y_list, dim = 1) #list를 tensor로 묶기 위해서 cat 함수 쓰는 것 decoder_y.shape = (문장 갯수, 단어 갯수(1개), 벡터 갯수)
        weight_list = torch.cat(weight_list, dim = 1)
        
        return decoder_y_list, decoder_hc, weight_list # decoder layer 2개 이상 묶어서 쓰기 위해서 hc값과, attention값도 return하기 위해 3개의 return 값
    def forward_cal(self, x, hc, encoder_output) : #rnn 블록하나 계산하는 함수
        h_for = hc[0][::2]
        h_back = hc[0][1::2]
        query = torch.cat([h_for, h_back], dim = -1).permute(1,0,2)
        
        context, weight = self.attention(query, encoder_output)
        
        x = self.embedding(x) # 벡터로 변환하고
        x = torch.cat([context, x], dim = -1) # attention 적용    
        x, hc = self.rnn(x, hc) # rnn 계산, hc값은 이전 hc계산 결과, hc값도 따로 입력합니다
        x = self.f(x) # 벡터 계산값 역산해서 다시 라벨로 변환
        return x, hc, weight
        
class Encoder_n_Decoder(nn.Module) : #encoder와 decoder를 하나로 묶는 새로운 신경망
    def __init__(self, encoder, decoder) :
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
    def forward(self, x, t = None) :
        y, hc = self.encoder(x)
        y, _, _ = self.decoder(y, hc, t)
        return y

In [7]:
encoder_file_name = 'data/encoder.pt'
decoder_file_name = 'data/decoder.pt'
pad_idx = 0

encoder = Encoder(len(kor_word), 100).to(device)
decoder = Decoder(len(eng_word), 100, 10).to(device)
loss_func = nn.CrossEntropyLoss(ignore_index=pad_idx)
encoder_optim = torch.optim.Adam(encoder.parameters(), lr = 0.02)
decoder_optim = torch.optim.Adam(decoder.parameters(), lr = 0.02)

epoch = 100
prev_acc = 0
cnt = 0

for e in range(epoch) :
    loss_sum = 0
    encoder.train() #dropout 켜주기
    decoder.train()
    for x, t in dataloader :
### y = F(x) (순전파)
        y, hc = encoder(x)
        y, _, _ = decoder(y, hc, t)
### 손실함수
        loss = loss_func(y.reshape(-1, y.shape[-1]) , t.reshape(-1)) #3차원을 2차원으로 펴주기
        loss_sum += loss.item()
### 역전파
        loss.backward()
        decoder_optim.step()
        encoder_optim.step()
        decoder_optim.zero_grad()
        encoder_optim.zero_grad()
    loss_sum /= len(dataloader) #평균 구하기
### 중간 점검
    correct = 0
    total = 0
    encoder.eval() #dropout 꺼주기
    decoder.eval()
    for x, t in test_dataloader :
        with torch.no_grad() :
            t_padding_mask = (t != pad_idx)
            y, hc = encoder(x)
            y, _, _ = decoder(y, hc)
            correct += (y.argmax(dim = -1)[t_padding_mask] == t[t_padding_mask]).sum().item()
        total += t_padding_mask.sum().item()
    acc = correct / total
### earlystopper
    if acc <= prev_acc :
        cnt += 1
    else :
        cnt = 0
        prev_acc = acc
        torch.save(encoder, encoder_file_name) #중간 저장
        torch.save(decoder, decoder_file_name)
    print(f"epoch {e+1} | loss {loss_sum} | acc {acc} | cnt {cnt}")
    if cnt >= 5 :
        print("train stopped")
        break

epoch 1 | loss 4.632988747683439 | acc 0.2544650751547303 | cnt 0
epoch 2 | loss 3.049969226663763 | acc 0.3032419687592101 | cnt 0
epoch 3 | loss 2.0987522797151046 | acc 0.3893309755378721 | cnt 0
epoch 4 | loss 1.4614329034631903 | acc 0.5264367816091954 | cnt 0
epoch 5 | loss 1.0429541035131975 | acc 0.6206896551724138 | cnt 0
epoch 6 | loss 0.742204714905132 | acc 0.7109048040082523 | cnt 0
epoch 7 | loss 0.5537535537372936 | acc 0.7720601237842617 | cnt 0
epoch 8 | loss 0.4128740971738642 | acc 0.8255820807544946 | cnt 0
epoch 9 | loss 0.3232164900411259 | acc 0.8704980842911877 | cnt 0
epoch 10 | loss 0.256750061024319 | acc 0.8871205422929561 | cnt 0
epoch 11 | loss 0.21348540417172693 | acc 0.9171529619805482 | cnt 0
epoch 12 | loss 0.18524594686248086 | acc 0.9228706159740643 | cnt 0
epoch 13 | loss 0.16675427095456558 | acc 0.9345711759504863 | cnt 0
epoch 14 | loss 0.14746667566624555 | acc 0.9407898614795166 | cnt 0
epoch 15 | loss 0.13898776783184572 | acc 0.9445328617742